In [ ]:
#Instruções do Mineda:
''''
Escolha uma tarefa de aprendizagem de máquina:
• Classificação
• Regressão
• Agrupamento (Clustering)
• Series Temporais
• Escolha, no mínimo, dois algoritmos para aplicar ao problema
• Exemplo: Para uma tarefa de classificação que pretende identificar que
alunos irão evadir da Fatec, você resolve utilizar Redes Neurais e Árvore de
Decisão
''''

#Vou usar serie temporal para prever os padrões de estilos que vão ficar em alta em qual periodo. 
#Identify trends in music over time using temporal series

In [1]:
#Pandas
import pandas as pd

import numpy as np

#Importando opendatasets para download do dataset do Kaggle
!pip install opendatasets
import opendatasets as od

# Configuração do matplotlib
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
%matplotlib inline

# Sklearn e Skforecast
from sklearn.ensemble import RandomForestRegressor

!pip install skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg

In [2]:
od.download("https://www.kaggle.com/datasets/thedevastator/popularity-of-spotify-top-tracks-by-genre")

Skipping, found downloaded files in "./popularity-of-spotify-top-tracks-by-genre" (use force=True to force download)


In [3]:
daily_charts = pd.read_csv('data/spotify_daily_charts.csv')
dc_artists = pd.read_csv('data/spotify_daily_charts_artists.csv')
dc_tracks = pd.read_csv('data/spotify_daily_charts_tracks.csv')
dc_tracks_rp = pd.read_csv('data/spotify_daily_charts_tracks_rec_pool.csv')

In [6]:
#Tratamento de dados
#Renomear a coluna de data

#Convertendo data de string para datetime e transforma em índice
daily_charts['date'] = pd.to_datetime(daily_charts['date'], format='%Y/%m/%d')
daily_charts = daily_charts.set_index('date')
daily_charts = daily_charts.asfreq('MS')
daily_charts = data.sort_index()

#Plota os dados
fig, ax = plt.subplots(figsize = (9, 4))
daily_charts['streams'].plot(ax=ax, label='streams')
daily_charts['artist'].plot(ax=ax, label='artist')
ax.legend();

KeyError: 'date'

In [24]:
#pd.set_option('display.max_rows', None)

In [25]:
#dc = daily_charts.head()
#dca = dc_artists.head()
#dct = dc_tracks.head()
#trp = dc_tracks_rp.head()

In [27]:
daily_charts.head()

,date,position,track_id,track_name,artist,streams
0,2017-01-01,1,0kN8xEmgMW9mh7UmDYHlJP,Versace on the Floor,Bruno Mars,185236
1,2017-01-01,2,5uCax9HTNlzGybIStD3vDh,Say You Won't Let Go,James Arthur,180552
2,2017-01-01,3,7BKLCZ1jbUBVqRi2FVlTVw,Closer,The Chainsmokers,158720
3,2017-01-01,4,2rizacJSyD9S1IQUxUxnsK,All We Know,The Chainsmokers,130874
4,2017-01-01,5,5MFzQMkrl1FOOng9tq6R9r,Don't Wanna Know,Maroon 5,129656


In [29]:
dc_artists.head()

,artist_id,artist_name,total_followers,genres,popularity
0,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,43236735,"['dance pop', 'pop']",87
1,4IWBUUAFIplrNtaOHcJPRM,James Arthur,10293374,"['pop', 'talent show', 'uk pop']",80
2,69GGBxA162lTqCwzJG5jLp,The Chainsmokers,19465938,"['dance pop', 'edm', 'electropop', 'pop', 'pop...",80
3,04gDigrS5kc9YWfZHwBETP,Maroon 5,37898474,['pop'],85
4,5p7f24Rk5HkUZsaS3BLG5F,Hailee Steinfeld,8049727,"['dance pop', 'pop', 'post-teen pop', 'uk pop']",72


In [31]:
dc_tracks_rp.head()

,track_id,track_name,artist_id,artist_name,album_id,duration,release_date,popularity,danceability,energy,...,valence,tempo,predicted_genre_id,predicted_genre,predicted_genre_prob,predicted_dance pop_prob,predicted_karaoke_prob,predicted_opm ost_prob,predicted_rnb_prob,predicted_rock_prob
0,43rA71bccXFGD4C8GOpIlN,I Forgot That You Existed,06HL4z0CvFAxyc27GXpf02,Taylor Swift,1NAmidJlEaVgA3MpcPFYGq,170640,2019-08-23,72,0.664,0.316,...,0.541,0.257286,3,rnb,0.987103,0.000850,0.011173,0.000470,0.987103,0.000405
1,5hTpBe8h35rJ67eAWHQsJx,Caroline,3Gm5F95VdRxW3mqCn8RPBJ,Aminé,3lajefIuUk4SfzqVBSJy8p,209640,2017-07-28,74,0.940,0.335,...,0.707,0.422371,3,rnb,0.984634,0.004066,0.010878,0.000102,0.984634,0.000320
2,5hTpBe8h35rJ67eAWHQsJx,Caroline,3Gm5F95VdRxW3mqCn8RPBJ,Aminé,3lajefIuUk4SfzqVBSJy8p,209640,2017-07-28,74,0.940,0.335,...,0.707,0.422371,3,rnb,0.984634,0.004066,0.010878,0.000102,0.984634,0.000320
3,5hTpBe8h35rJ67eAWHQsJx,Caroline,3Gm5F95VdRxW3mqCn8RPBJ,Aminé,3lajefIuUk4SfzqVBSJy8p,209640,2017-07-28,74,0.940,0.335,...,0.707,0.422371,3,rnb,0.984634,0.004066,0.010878,0.000102,0.984634,0.000320
4,5hTpBe8h35rJ67eAWHQsJx,Caroline,3Gm5F95VdRxW3mqCn8RPBJ,Aminé,3lajefIuUk4SfzqVBSJy8p,209640,2017-07-28,74,0.940,0.335,...,0.707,0.422371,3,rnb,0.984634,0.004066,0.010878,0.000102,0.984634,0.000320


In [35]:
dc_tracks.head()

,track_id,track_name,artist_id,artist_name,album_id,duration,release_date,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0kN8xEmgMW9mh7UmDYHlJP,Versace on the Floor,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,4PgleR09JVnm3zY1fW3XBA,261240,2016-11-17,75,0.578,0.574,2.0,-6.209,1.0,0.0454,0.196,0.00000,0.0830,0.301,174.152
1,5uCax9HTNlzGybIStD3vDh,Say You Won't Let Go,4IWBUUAFIplrNtaOHcJPRM,James Arthur,7oiJYvEJHsmYtrgviAVIBD,211466,2016-10-28,85,0.358,0.557,10.0,-7.398,1.0,0.0590,0.695,0.00000,0.0902,0.494,85.043
2,7BKLCZ1jbUBVqRi2FVlTVw,Closer,69GGBxA162lTqCwzJG5jLp,The Chainsmokers,0rSLgV8p5FzfnqlEk4GzxE,244960,2016-07-29,84,0.748,0.524,8.0,-5.599,1.0,0.0338,0.414,0.00000,0.1110,0.661,95.010
3,2rizacJSyD9S1IQUxUxnsK,All We Know,69GGBxA162lTqCwzJG5jLp,The Chainsmokers,0xmaV6EtJ4M3ebZUPRnhyb,194080,2016-09-29,70,0.662,0.586,0.0,-8.821,1.0,0.0307,0.097,0.00272,0.1150,0.296,90.000
4,5MFzQMkrl1FOOng9tq6R9r,Don't Wanna Know,04gDigrS5kc9YWfZHwBETP,Maroon 5,0fvTn3WXF39kQs9i3bnNpP,214480,2016-10-11,0,0.783,0.623,7.0,-6.126,1.0,0.0800,0.338,0.00000,0.0975,0.447,100.048


In [34]:
#Verificando as datas do daily charts e tirando os itens duplicados.
daily_charts['date'].drop_duplicates()

0         2017-01-01
200       2017-01-02
400       2017-01-03
600       2017-01-04
800       2017-01-05
1000      2017-01-06
1200      2017-01-07
1400      2017-01-08
1600      2017-01-09
1800      2017-01-10
2000      2017-01-11
2200      2017-01-12
2400      2017-01-13
2600      2017-01-14
2800      2017-01-15
3000      2017-01-16
3200      2017-01-17
3400      2017-01-18
3600      2017-01-19
3800      2017-01-20
4000      2017-01-21
4200      2017-01-22
4400      2017-01-23
4600      2017-01-24
4800      2017-01-25
5000      2017-01-26
5200      2017-01-27
5400      2017-01-28
5600      2017-01-29
5800      2017-01-30
6000      2017-01-31
6200      2017-02-01
6400      2017-02-02
6600      2017-02-03
6800      2017-02-04
7000      2017-02-05
7200      2017-02-06
7400      2017-02-07
7600      2017-02-08
7800      2017-02-09
8000      2017-02-10
8200      2017-02-11
8400      2017-02-12
8600      2017-02-13
8800      2017-02-14
9000      2017-02-15
9200      2017-02-16
9400      201

In [7]:
print('Registros Rock Playlist Data: '+str(len(rock_playlist_data_csv)))

Registros Rock Playlist Data: 97


In [8]:
print('Registros Rock Track: '+str(len(rock_playlist_track_csv)))

Registros Rock Track: 2132


In [9]:
print('Registros Track Data: '+str(len(rock_tracks_data_csv)))

Registros Track Data: 2131
